In [1]:
import numpy as np
import mido

In [2]:
data = np.load("data.npy")

In [210]:
def convert_note(note):
    notes = "C,Db,D,Eb,E,F,Gb,G,Ab,A,Bb,B".split(",")
    numbers = list(range(60,72))  
    try:
        octave = int(note[-1])
        note_idx = notes.index(note[:-1])
    except:
        octave = 4
        note_idx = notes.index(note)
    offset = (octave-4)*12    
    return numbers[note_idx] + offset

def map_note(value, root=60):
    """
    value: a numeric value ranged from 0 to 1
    root: the root note used as the reference
    """
    disct = (value//0.1).astype(np.int32)
    major_scale = [2,2,1,2,2,2,1]    
    scale = np.cumsum([*(major_scale*2)])
    
    return scale[disct] + root

In [182]:
def map_chord(value, ref):
    value0 = (value-np.mean(value))/np.std(value)
    value = np.maximum(np.minimum(value0, 3), -3) + 3
    disct = value.astype(np.int32)
    intervals = np.array([5, 7, 4, 4, 7, 7, 5])
    
    return ref-intervals[disct]

def map_duration(value):
    value0 = (value-np.mean(value))/np.std(value)
    value = np.maximum(np.minimum(value0, 3), -3) + 3
    duration = np.array(
        [0.5]*3 + [1]*2 + [1.5]*2
    )
    disct = value.astype(np.int32)
    return duration[disct], disct

In [183]:
def map_track1_data(melody_data, chord_data, duration_data, root):
    notes = map_note(melody_data, root)
    chord = map_chord(chord_data, notes)
    duration, value0 = map_duration(duration_data)
    track = [notes, chord, duration]
    return track

In [184]:
def map_track2_data(melody_data, chord_data, duration_data, ref_notes, base_offset=-12):
    notes = map_chord(melody_data, ref_notes+base_offset)
    chord = map_chord(chord_data, notes)
    duration, value0 = map_duration(duration_data)
    track = [notes, chord, duration]
    return track

## playing

In [185]:
import time

In [186]:
def make_piano_track(track_data, ticks_per_beat, bpm=120, velocity=80, program=1):    
    track = mido.MidiTrack()
    track.append(mido.Message("program_change", program=program))
    for note1, note2, beat in zip(track_data[0], track_data[1], track_data[2]):
        ticks = int(mido.second2tick((60/bpm)*beat, ticks_per_beat, mido.bpm2tempo(120)))
        msg11 = mido.Message('note_on', note=note1, velocity=velocity, time=0)
        msg12 = mido.Message('note_on', note=note2, velocity=velocity, time=0)
        msg21 = mido.Message('note_off', note=note1, velocity=velocity, time=ticks)
        msg22 = mido.Message('note_off', note=note2, velocity=velocity, time=0)            
        track.append(msg11)        
        track.append(msg12)
        track.append(msg21)        
        track.append(msg22)        
    return track

In [187]:
def make_string_track(track_data, ticks_per_beat, bpm=120, velocity=80, program=1):    
    track = mido.MidiTrack()
    track.append(mido.Message("program_change", program=program, channel=2))
    mask_gate = 0
    acc_ticks = 0
    playing_note = 0
    for note1, beat in zip(track_data[0], track_data[2]):
        ticks = int(mido.second2tick((60/bpm)*beat, ticks_per_beat, mido.bpm2tempo(120)))
        acc_ticks += ticks        
        if mask_gate == 0:
            msg11 = mido.Message('note_on', note=note1, velocity=velocity, time=0, channel=2)
            track.append(msg11)
            playing_note = note1 
            mask_gate += 1
        elif mask_gate == 2:
            msg21 = mido.Message('note_off', note=playing_note, velocity=velocity, time=acc_ticks, channel=2)
            track.append(msg21)
            mask_gate = 0
            acc_ticks = 0        
        else:
            mask_gate += 1
    track.append(mido.Message('note_off', note=playing_note, velocity=velocity, time=acc_ticks, channel=2))
    return track

In [188]:
track1 = map_track1_data(data[0,:], data[1,:], data[2,:], root=67)
track2 = map_track2_data(data[3,:], data[4,:], data[5,:], ref_notes=track1[0])

In [189]:
def make_clip(length, track):
    return [x[:length] for x in track]
value_counts = 112
track1 = make_clip(value_counts, track1)
track2 = make_clip(value_counts, track2)

In [190]:
mid = mido.MidiFile()
bpm = 112
tpb = 64
mid.ticks_per_beat=tpb
mid.tracks.append(make_piano_track(track1, ticks_per_beat=tpb, bpm=bpm, program=1))
mid.tracks.append(make_piano_track(track2, ticks_per_beat=tpb, bpm=bpm, velocity=64, program=1))
mid.tracks.append(make_string_track(track2, ticks_per_beat=tpb, bpm=bpm, velocity=64, program=9))
mid.save("test.mid")

In [191]:
with mido.open_output() as port:
    for msg in mido.MidiFile('test.mid').play():
        port.send(msg)

In [53]:
port.send(mido.Message("stop"))

In [33]:
len(track2[2])

768

In [41]:
[x.time for x in mid.tracks[2] if x.type=="note_off"]

[128, 96, 128, 128, 96, 128, 96]

In [40]:
mid.tracks[2][1].type

'note_on'

In [64]:
list(mid.tracks[2])

[<message program_change channel=2 program=44 time=0>,
 <message note_on channel=2 note=53 velocity=80 time=0>,
 <message note_off channel=2 note=53 velocity=80 time=128>,
 <message note_on channel=2 note=67 velocity=80 time=0>,
 <message note_off channel=2 note=67 velocity=80 time=96>,
 <message note_on channel=2 note=60 velocity=80 time=0>,
 <message note_off channel=2 note=60 velocity=80 time=128>,
 <message note_on channel=2 note=65 velocity=80 time=0>,
 <message note_off channel=2 note=65 velocity=80 time=128>,
 <message note_on channel=2 note=48 velocity=80 time=0>,
 <message note_off channel=2 note=48 velocity=80 time=96>,
 <message note_on channel=2 note=46 velocity=80 time=0>,
 <message note_off channel=2 note=46 velocity=80 time=128>,
 <message note_on channel=2 note=46 velocity=80 time=0>,
 <message note_off channel=2 note=46 velocity=80 time=96>]